In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, DIAMETER_STD, TEMPERATURE_STD, IN_PROFILE
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool
import os

In [2]:
import pyroll.basic as pr
#import pyroll.jmak_recrystallization

In [3]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_input_results.csv"

In [4]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

In [5]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

In [6]:
def worker(draw: DrawInput):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [7]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:42<00:00, 23.35it/s]


In [8]:
df = results |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                  roll_force                                \
     diameter  temperature             R1             R2             R3   
0    0.050390  1465.501074  280462.678271  142200.363254  254379.221828   
1    0.050772  1399.842730  307964.563537  167315.135400  286981.629856   
2    0.051325  1404.773754  331497.807637  171519.196663  284989.149340   
3    0.049996  1429.311189  293837.398229  157669.346430  270434.703654   
4    0.050787  1501.640547  243754.449174  135417.735475  239468.516829   
..        ...          ...            ...            ...            ...   
995  0.050207  1455.934724  283406.586908  142502.207671  258493.353240   
996  0.049552  1435.284008  281654.737797  154132.750537  266771.130061   
997  0.047715  1494.210175  227694.157857  126518.070314  236869.476847   
998  0.049303  1390.175463  259987.589675  155948.129761  290046.202187   
999  0.048064  1495.454747  231720.737676  125992.968474  237083.797070   

                                                                               \
                R4             R5             R6             R7            R8   
0    119739.591401  163037.743090   82455.852085  163991.947463  82203.224159   
1    132735.790734  178327.531986   89146.188744  174925.613107  73819.521823   
2    131453.848185  176831.456288   88475.381057  173826.995351  73432.122996   
3    126663.551955  171192.554003  102724.693640  183452.367180  75974.072670   
4    113306.603729  155426.375968   79097.596826  158459.284900  68021.195938   
..             ...            ...            ...            ...           ...   
995  121580.068588  165208.278155   83414.738716  165567.970534  82232.166614   
996  125551.742691  169879.397225  101585.769052  180730.751556  80625.068988   
997  114758.075181  157107.968554   79945.375975  159900.878030  68541.932433   
998  135358.323089  181377.871481   90531.634082  177198.574128  74622.344848   
999  114525.964329  156838.468788   79811.234230  159673.512493  68459.988841   

     ... out_recrystallized_fraction                                          \
     ...                          R5        R6   R7        R8        R9  R10   
0    ...                         1.0  1.000000  1.0  0.593833  0.999613  1.0   
1    ...                         1.0  0.997628  1.0  1.000000  1.000000  1.0   
2    ...                         1.0  0.987047  1.0  0.999998  1.000000  1.0   
3    ...                         1.0  0.136063  1.0  0.920844  1.000000  1.0   
4    ...                         1.0  1.000000  1.0  1.000000  1.000000  1.0   
..   ...                         ...       ...  ...       ...       ...  ...   
995  ...                         1.0  0.998978  1.0  0.028495  0.998971  1.0   
996  ...                         1.0  0.014523  1.0  0.775001  1.000000  1.0   
997  ...                         1.0  1.000000  1.0  1.000000  1.000000  1.0   
998  ...                         1.0  0.999942  1.0  1.000000  1.000000  1.0   
999  ...                         1.0  1.000000  1.0  1.000000  1.000000  1.0   

                              
      F1   F2   F3        F4  
0    1.0  1.0  1.0  0.999996  
1    1.0  1.0  1.0  1.000000  
2    1.0  1.0  1.0  1.000000  
3    1.0  1.0  1.0  0.999999  
4    1.0  1.0  1.0  0.999993  
..   ...  ...  ...       ...  
995  1.0  1.0  1.0  0.999997  
996  1.0  1.0  1.0  0.999998  
997  1.0  1.0  1.0  0.999994  
998  1.0  1.0  1.0  1.000000  
999  1.0  1.0  1.0  0.999994  

[1000 rows x 170 columns]

In [9]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)